# CNN 1D:

In [ ]:
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalAveragePooling1D, Input, Dense, Dropout

**Data Loading and Feature Extraction:**

In [ ]:
# --- Feature Extraction Functions ---
#Need to develop your own feature extraction functions based on a method called 'Classification of micro-Doppler radar hand-gesture signatures by means of Chebyshev moments
# --- Data Loading  ---

base_path='/Users/beno/Desktop/TU DELFT/ML for EE/WICOS' #A CHANGER!
people = ['Person A', 'Person B', 'Person C', 'Person D', 'Person E', 'Person F']  
gestures = ['click', 'pinch', 'swipe', 'wave']

X_all = [] 
y_all = []

print("Loading and extracting 1D envelope features...")
for person in people:
    for gesture in gestures:
        gesture_path = os.path.join(base_path, person, gesture)
        if os.path.exists(gesture_path):
            files = [f for f in os.listdir(gesture_path) if f.endswith('.csv')]
            for f in files:
                df = pd.read_csv(os.path.join(gesture_path, f), dtype=np.float32)
                
                #  Preprocess spectrogram
                matrix, freqs = preprocess_spectrogram(df)
                
                #  Extract 1D Envelopes
                eU, eL = extract_envelopes(matrix, freqs)
                
                # Build augmented feature vector (as per paper)
                feature_vector = build_feature_vector(eU, eL)
                
                # Append 1D vector and label
                X_all.append(feature_vector)
                y_all.append(gesture)
        else:
            print(f"Warning: Data directory not found for {person}/{gesture}")

print(f"Loaded {len(X_all)} total samples.")

**Data Preparation for 2D CNN:**

In [ ]:
# Pad all 1D feature vectors to be the same size
X_padded = pad_to_max_length(X_all)

# Convert labels to one-hot encoding
label_encoder = LabelEncoder()
y_int = label_encoder.fit_transform(y_all)
onehot_encoder = OneHotEncoder(sparse_output=False)
y_oh = onehot_encoder.fit_transform(y_int.reshape(-1, 1))

# Add a "channels" dimension for the 1D CNN
# Shape becomes (num_samples, max_length, 1)
X_cnn_1d = np.expand_dims(X_padded, axis=2)

# Get input shape and number of classes
input_shape = X_cnn_1d.shape[1:]
num_classes = y_oh.shape[1]

# Create Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(
    X_cnn_1d, y_oh, test_size=0.2, random_state=42, stratify=y_all
)

print(f"X_train shape for 1D CNN: {X_train.shape}")
print(f"y_train shape for 1D CNN: {y_train.shape}")
print(f"Input shape for 1D CNN: {input_shape}")
print(f"Number of classes: {num_classes}")

**Defining the CNN model:**

In [ ]:
model_1d = Sequential()

# Input Layer - shape is (timesteps, features=1)
model_1d.add(Input(shape=input_shape))

# Convolutional Block 1
model_1d.add(Conv1D(filters=16, kernel_size=3, activation='relu', padding='same'))
model_1d.add(MaxPooling1D(pool_size=2))

# Convolutional Block 2
model_1d.add(Conv1D(filters=32, kernel_size=3, activation='relu', padding='same'))
model_1d.add(MaxPooling1D(pool_size=2))

# Convolutional Block 3
model_1d.add(Conv1D(filters=64, kernel_size=3, activation='relu', padding='same'))

# Use GlobalAveragePooling1D to reduce parameters and handle sequences
model_1d.add(GlobalAveragePooling1D())

# Dropout for regularization
model_1d.add(Dropout(0.5))

# Output layer
model_1d.add(Dense(num_classes, activation='softmax'))

# Compile the model
model_1d.compile(loss='categorical_crossentropy',
                 optimizer='adam',
                 metrics=['accuracy'])

model_1d.summary()

**Model Training:**

In [ ]:
# --- MODEL TRAINING ---
print("\nTraining 1D CNN model...")
history = model_1d.fit(X_train, y_train,
                       epochs=50,  # This simpler model can train for more epochs
                       batch_size=32,
                       validation_data=(X_test, y_test))

# --- MODEL EVALUATION ---
print("\nEvaluating 1D CNN model...")
y_pred_probs = model_1d.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)
y_test_labels = np.argmax(y_test, axis=1)

accuracy = accuracy_score(y_test_labels, y_pred)
print(f"Test Accuracy: {accuracy * 100:.2f}%")
print("\nClassification Report:")
print(classification_report(y_test_labels, y_pred, target_names=label_encoder.classes_))